In [1]:
#####################################################################
# Mandatory : Run this cell and restart the notebook kernel right after
#####################################################################
!pip install torch==1.11 transformers stanza

     |████████████████████████████████| 574 kB 22.8 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 23.2 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193002 sha256=f2ba39c5bfc065aae279bed1af2dbc61228ec33cdc462937c87bff48a815fdbb
  Stored in directory: /home/pbeuran/.cache/pip/wheels/23/a5/a8/e74bad1ceced228b6ae94dcbacc5c67df6486fd1620714e7d1
Successfully built emoji


In [2]:
%cd /content

[Errno 2] No such file or directory: '/content'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [15]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
%cd deepqa/notebooks/

[Errno 2] No such file or directory: 'deepqa/notebooks/'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [4]:
%%bash
cd ..
curl -O https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
unzip -o glove.6B.zip -d word_encoders_configs
rm glove.6B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  822M  100  822M    0     0  4124k      0  0:03:24  0:03:24 --:--:-- 4146k


Archive:  glove.6B.zip
  inflating: word_encoders_configs/glove.6B.50d.txt  
  inflating: word_encoders_configs/glove.6B.100d.txt  
  inflating: word_encoders_configs/glove.6B.200d.txt  
  inflating: word_encoders_configs/glove.6B.300d.txt  


In [2]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

2022-07-30 00:36:05.538503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-30 00:36:05.538577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
import sys
import os
import pathlib

import json

import pandas as pd
import numpy as np
import torch

from tqdm import tqdm

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import preprocessing, tokenizer, model, wrapper, loss, metrics, utils

2022-08-05 11:49:00.872813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 11:49:00.872914: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [3]:
train_tcqa = preprocessing.tabularize_squad11_data(train_data)
dev_tcqa = preprocessing.tabularize_squad11_data(dev_data)

In [4]:
import re
from spacy.lang.en import English
import pickle
from deepqa.utils import char_ranges_to_token_ranges

class GLoVETokenizer():

    def __init__(self, corpus="6B", tokenize_char=False):

        self.word_tokenizer = English().tokenizer
        with open(f"../word_encoders_configs/glove.{corpus}.50d.txt", "r") as f:
            lines = f.readlines()
        lines = [line.replace("\n", "").split(" ") for line in lines]
        word_vocab = [line[0] for line in lines]
        self.word_vocab_id_mapping = {**dict(zip(word_vocab, range(len(word_vocab)))),
                                      **{"": len(word_vocab) + 1}}
        self.word_vocab_len = len(self.word_vocab_id_mapping)
        self.tokenize_char = tokenize_char
        if tokenize_char:
            char_token_vocab_path = "../tokenizers_configs"
            char_vocab = pickle.load(open(f"{char_token_vocab_path}/bert-base-uncased.ctk", "rb"))
            self.char_vocab_len = len(char_vocab)
            self.char_vocab_id_mapping = dict(zip(char_vocab, range(len(char_vocab))))


    def __call__(self, texts, tokens_max_length, token_max_length, **kwargs):

        tokenize_result = dict()

        texts = [re.sub(r" {2,}", "", text) for text in texts]

        inputs_tokens = [list(doc) for doc in self.word_tokenizer.pipe(texts)]

        inputs_truncated = [len(input_tokens) > tokens_max_length
                            for input_tokens in inputs_tokens]
        inputs_tokens = [input_tokens[:min(len(input_tokens), tokens_max_length)] 
                         for input_tokens in inputs_tokens]
        inputs_low_pad_tokens = [[str(input_token).lower() 
                                  for input_token in input_tokens] +
                                 (tokens_max_length - len(input_tokens)) * [""]
                                for input_tokens in inputs_tokens]
        tokenize_result["truncated"] = torch.tensor(inputs_truncated)

        inputs_ids = [[self.word_vocab_id_mapping.get(input_low_pad_token, 
                                                      self.word_vocab_len)
                       for input_low_pad_token in input_low_pad_tokens]
                      for input_low_pad_tokens in inputs_low_pad_tokens]
        tokenize_result["input_ids"] = torch.tensor(inputs_ids, dtype=torch.int32, 
                                                    device="cpu")

        offset_mappings = [[(input_token.idx, input_token.idx + len(input_token))
                            for input_token in input_tokens] +
                           ((tokens_max_length - len(input_tokens)) * [(0, 0)])
                           for input_tokens in inputs_tokens]
        tokenize_result["offset_mapping"] = torch.tensor(offset_mappings, dtype=torch.int32, 
                                                         device="cpu")

        if self.tokenize_char:
            inputs_tokens_char_ids = (
                [[[self.char_vocab_id_mapping.get(char, self.char_vocab_len) 
                   for char in list(input_low_pad_token)] +
                   ((token_max_length - len(input_low_pad_token)) *
                    [self.char_vocab_len + 1])
                  for input_low_pad_token in input_low_pad_tokens]
                 for input_low_pad_tokens in inputs_low_pad_tokens]
            )
            tokenize_result["inputs_char_ids"] = torch.tensor(inputs_tokens_char_ids, 
                                                              dtype=torch.int32, 
                                                              device="cpu")
        
        return tokenize_result

    def tokenize_qa_data(self, contexts, queries, answers,
                         context_max_length, query_max_length, token_max_length):
        contexts_tokens = self(contexts, context_max_length, token_max_length)
        queries_tokens = self(queries, query_max_length, token_max_length)
        answers_chars_range = [(answer["answer_start"], 
                                answer["answer_start"] + len(answer["text"]))
                               for answer in answers]
        answers_tokens_range = char_ranges_to_token_ranges(answers_chars_range, 
                                                           contexts_tokens["offset_mapping"],
                                                           context_max_length)
        return contexts_tokens, queries_tokens, answers_tokens_range

2022-08-05 11:49:04.939764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-05 11:49:04.939860: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-05 11:49:04.939898: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-6LCD53E): /proc/driver/nvidia/version does not exist


In [5]:
glove_tokenizer = GLoVETokenizer(tokenize_char=True)

context_max_length = 300
query_max_length = 40
token_max_length = 40

train_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(train_tcqa[1:]), context_max_length, query_max_length, token_max_length
)
dev_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(dev_tcqa[1:]), context_max_length, query_max_length, token_max_length
)

In [6]:
class QADataset(torch.utils.data.Dataset):

    def __init__(self, contexts_tokens, queries_tokens, answers_tokens_range,
                 device = "cpu"):

        to_remove = ~(contexts_tokens["truncated"] | queries_tokens["truncated"])
        self.contexts_tokens = {k:v[to_remove] for k,v in contexts_tokens.items()}
        self.queries_tokens = {k:v[to_remove] for k,v in queries_tokens.items()}
        self.answers_tokens_range = answers_tokens_range[to_remove]
        self.device = device

    def __getitem__(self, index):
        return {"contexts_tokens": {k:v[index].to(self.device)
                                    for k,v in self.contexts_tokens.items()},
                "queries_tokens": {k:v[index].to(self.device)
                                    for k,v in self.queries_tokens.items()},
                "answers_tokens_range": self.answers_tokens_range[index].to(self.device)}

    def __len__(self):
        return len(self.answers_tokens_range)

In [7]:
class GLoVEWordEncoder(torch.nn.Module):

    def __init__(self, corpus="6B", embedding_size=50, char_encoder=None):
        
        super(GLoVEWordEncoder, self).__init__()
        with open(f"../word_encoders_configs/glove.{corpus}.{embedding_size}d.txt", "r") as f:
            lines = f.readlines()
        lines = [line.replace("\n", "").split(" ") for line in lines]
        word_embeddings = torch.zeros((len(lines) + 2, embedding_size))
        word_embeddings[:len(lines), :] = torch.tensor(
            [[float(embedding) for embedding in line[1:]] for line in lines]
        )
        self.word_embeddings = torch.nn.Embedding.from_pretrained(word_embeddings,
                                                                  padding_idx=len(lines) + 1)
        self.char_encoder = char_encoder

    def forward(self, tokens):

        word_embeddings = self.word_embeddings(tokens["input_ids"])
        return word_embeddings

    def output_shape(self):
        output_shape = self.word_embeddings.weight.shape[1]
        if self.char_encoder is not None:
            output_shape = output_shape + self.char_encoder.output_shape()
        return self.word_embeddings.weight.shape[1]


class CharCNN(torch.nn.Module):

    def __init__(self,
                 char_vocab_len,
                 char_embedding_size, 
                 outs_channels,
                 kernel_sizes):

        super(CharCNN, self).__init__()
        self.char_encoder = torch.nn.Embedding(char_vocab_len + 3, 
                                               char_embedding_size, 
                                               char_vocab_len + 2)
        self.in_channels = char_embedding_size
        self.outs_channels = outs_channels
        self.kernel_sizes = kernel_sizes
        self.conv1d_list = list()
        for i, kernel_size in enumerate(kernel_sizes):
            conv1d = torch.nn.Conv1d(char_embedding_size, outs_channels[i], kernel_size)
            setattr(self, f"conv{i+1}", conv1d)
            self.conv1d_list.append(conv1d)

    def forward(self, char_tokens):

        char_embeddings = self.char_encoder(char_tokens)

        batch_size = char_embeddings.shape[0]
        token_seq_size = char_embeddings.shape[1]
        token_char_seq_size = char_embeddings.shape[2]
        features_size = char_embeddings.shape[3]
        
        rav_char_embeddings = (char_embeddings.transpose(3, 2)
                                              .reshape(batch_size * token_seq_size,
                                                       features_size,
                                                       token_char_seq_size))
        rav_char_embeddings = torch.cat(
            [torch.nn.Tanh()(conv1d(rav_char_embeddings)).max(dim=2)[0]
             for conv1d in self.conv1d_list],
             dim=1
        ).reshape(batch_size, token_seq_size, -1)
        return rav_char_embeddings

    def output_shape(self):
        return sum([conv1d.out_channels for conv1d in self.conv1d_list])

In [8]:
char_cnn = CharCNN(1371, 16, [100], [5])

glove_word_encoder = GLoVEWordEncoder(embedding_size=200, char_encoder=char_cnn)
contextual_embedding_size = 100

In [9]:
bidaf = model.BiDAF(glove_word_encoder, contextual_embedding_size, 0.2)

In [10]:
train_dataset = QADataset(*train_tokens_cqa, "cuda")
train_loader = torch.utils.data.DataLoader(train_dataset, 64)

dev_dataset = QADataset(*dev_tokens_cqa, "cuda")
dev_loader = torch.utils.data.DataLoader(dev_dataset, 64)

bidaf_trainer = wrapper.QATrainWrapper(bidaf)
bidaf_trainer.train(train_loader, 
                    epochs=16, 
                    loss=loss.bi_cross_entropy,
                    val_data_iter=dev_loader)

  0%|          | 0/1345 [00:00<?, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx